# The idea of the project is to create DB of math problems for schoolkids.

## Reasons:

### $\bullet$ This speeds up the process of learning by preparing solutions and explanations for all school themes.
### $\bullet$ Reduces the time for preparation and allowing to create more data in the same time.

### Main difficulties are:

#### $\bullet$ I need to create plenty of problems from school course.
#### $\bullet$ Plenty of time is needed even for one solution to be created.
#### $\bullet$ When created, problems must be selected after a thorough filtration.

## Part 1:

Problems were actively created in 2020-2021.

I am very grateful to those who helped me type the main array of tasks at the start and look for errors in the tasks and their design.

Without them, this project would not have had a chance to be realized. In total, over 1800 problems and most of the solutions were written in $\LaTeX$.

## Part 2:

###### To implement the selection of several problems from the array, the problems will be stored in the SQL database.
But each math problem has 8 parameters and we need a bit of preprocessing...

1) Theme of the problem - approximate and doesn't show all the methods used in the problem. Themes from standard classbooks are used as a basis.

2) Class (as one of the main ways to filter problem later)

3) Topics/methods (the plan is to get an algorithm to quickly select all related problems from base ones to the difficult examples from high grades)

4) Difficulty of the current problem (it was hard to differentiate, hence this is a binary easy-difficult feature)

5) Condition of the problem itself (including images and tables, if present)

6) Expanded solution of the problem

7) Answer for the problem

8) Hint for the solution (used to give the students a chance to solve more problems)

## 2.1. Strings manipulation in Python

In [1]:
import os
import re
import pandas as pd

In [2]:
source = open("C://Users//novle//AppData//Local//MiKTeX//problems.txt", "r", encoding="utf8") # closed source
# set of problems in format \begin{problem}{}{}{}{}{}{}{}{}\end{problem} is taken
text = source.read() # temporarily import all into one large string
recurr_pattern = re.escape('\n\end{problem}\n\n\\begin{problem}') # pattern for preprocessing data before import into sql

In [3]:
problems_list = re.split(recurr_pattern, text) # each element is one problem, but extra cleaning needed:
# len(problems_list)
problems_list[0] = '{Делители и кратные.}{6.1.1}{6K \\textcolor{olive}{\\textbf{$\\spadesuit$}}}{(лёгкая)}\n{Сколько существует чисел от $1$ до $41$, которые делятся на $7$, но не делятся на $2$?}\n{Выпишем те числа, которые делятся на 7: это 7, 14, 21, 28, 35 (следующее число~--- 42~--- уже больше 41). Выберем из них те, которые не делятся на 2, то есть нечётные. Остаются 7, 21, 35: 14 и 28~--- чётные.}\n{Таких чисел три: это 7, 21 и 35.}{Для начала, а сколько всего чисел, которые делятся на 7?}'
problems_list[1806] = '{Вывод некоторых формул с помощью интегрирования.}{11.3.6}{10A}{(лёгкая)}\n{Радиусы трёх сплошных металлических шариков равны 3, 4 и 5 см соответственно. Эти шарики расплавили, и из получившегося металла отлили новый сплошной шар. Чему будет равен его радиус?}\n{НаписанноеРешение}\n{ВерныйОтвет}{Подсказка}'

In [4]:
def splitter(problem): # reformats problems.txt into one array
    countparens = 0 # counts open and closed parenthesis like Latex; precisely countparens = open - closed ones
    endpoints = [] # here we will store borders of fields
    mas = []
    for n in range(len(problem)):
        if problem[n] == '}' and problem[n-1] != '\\': # new parenthesis and not in a \} situation -> -1
            countparens -= 1
        if problem[n] == '{' and problem[n-1] != '\\': # new parenthesis and not in a \{ situation -> +1
            countparens += 1
        if countparens == 1 and problem[n] == '{' and problem[n-1] != '\\':
            endpoints.append(n)                        # the outer area; remember the indices which give the left borders
        if (countparens == 0) and problem[n] == '}' and problem[n-1] != '\\':
            endpoints.append(n)                        # the outer area; remember the indices which give the right borders
    for i in range(int(len(endpoints)/2)):
        mas.append(problem[(endpoints[2*i]+1) : endpoints[2*i+1]])   # write all 8 fields into the array and return it
    return mas # result is m = [theme, theme_code, extra_info, difficulty, condition, solution, answer, hint]

## 2.2. (example) The unprocessed data and the resulting array

In [5]:
# unprocessed data example: { and } are used in Latex for \begin{problem}...\end{problem} environment
print(problems_list[1234])

{Системы уравнений и неравенств с двумя переменными.}{9.3.3}{Метод подстановки}{(лёгкая)}
{Решить систему уравнений $\displaystyle\;\left\{
\begin{aligned}
    \: (x - 4)(y + 3) &= 0\\
    \: 4y - 3x =&\; 12.
\end{aligned}\right.$}
{$4y - 3x =\; 12$. Отсюда следует, что $\displaystyle y=\frac{12+3x}{4}$. Подставим это выражение в первое уравнение системы $(x - 4)(y + 3) = 0$ и получим $\displaystyle (x - 4)(\frac{12+3x}{4} + 3) = 0$.
Следовательно, $\displaystyle(x - 4)(\frac34 x + 6) = 0 \;\Rightarrow\; x_{1,2} = -8;4$. Тогда $\displaystyle y_1 = \frac{12-24}{4} = -3$; $\displaystyle y_2 = \frac{12+12}{4} = 6$, значит ответом являются пары чисел $(-8;-3)$ и $(4;6)$.}
{Система уравнений имеет два решения: $(-8;-3)$ и $(4;6)$.}{Вырази $y$ из второго уравнения системы и подставь в первое.}


In [6]:
# resulting array:
contents = ['Theme: ', 'Class: ', 'Methods: ', 'Difficulty: ', 'Condition: ', 'Solution: ', 'Answer: ', 'Hint: ']
problem_ex = splitter(problems_list[1234])
for i in range(8):
    print(contents[i], problem_ex[i], '\n')

Theme:  Системы уравнений и неравенств с двумя переменными. 

Class:  9.3.3 

Methods:  Метод подстановки 

Difficulty:  (лёгкая) 

Condition:  Решить систему уравнений $\displaystyle\;\left\{
\begin{aligned}
    \: (x - 4)(y + 3) &= 0\\
    \: 4y - 3x =&\; 12.
\end{aligned}\right.$ 

Solution:  $4y - 3x =\; 12$. Отсюда следует, что $\displaystyle y=\frac{12+3x}{4}$. Подставим это выражение в первое уравнение системы $(x - 4)(y + 3) = 0$ и получим $\displaystyle (x - 4)(\frac{12+3x}{4} + 3) = 0$.
Следовательно, $\displaystyle(x - 4)(\frac34 x + 6) = 0 \;\Rightarrow\; x_{1,2} = -8;4$. Тогда $\displaystyle y_1 = \frac{12-24}{4} = -3$; $\displaystyle y_2 = \frac{12+12}{4} = 6$, значит ответом являются пары чисел $(-8;-3)$ и $(4;6)$. 

Answer:  Система уравнений имеет два решения: $(-8;-3)$ и $(4;6)$. 

Hint:  Вырази $y$ из второго уравнения системы и подставь в первое. 



## 2.3. Extra array preprocessing in Python

In [7]:
#get the list of all themes available (at the moment, source = txt file, later will be changed to sql)
def get_unique_themes(problems): 
    unique = []
    for i in range(len(problems)):
        if splitter(problems_list[i])[0] in unique:
            continue
        else:
            unique.append(splitter(problems_list[i])[0])
    return unique

themes = get_unique_themes(problems_list)
# len(themes)
# themes

In [8]:
themes6 = themes[0:41]
# print(themes6)
themes7 = themes[41:75]
# print(themes7)
themes8 = themes[75:106]
# print(themes8)
themes9 = themes[106:135]
# print(themes9)
themes10 = themes[135:155]
# print(themes10)
themes11 = themes[155:176]
# print(themes11)
themes_choose = [themes6, themes7, themes8, themes9, themes10, themes11]

In [9]:
def get_problem_class(problem):
    clas = re.split('\.', problem[1])[0] # returns class (default for the theme and this problem)
    return clas

def get_problem_diff(problem): # will be stored as binary true/false variable
    if problem[3] == '*':
        return '1' # difficult one
    else:
        return '0' # easy one

In [10]:
total = [] # rebuilding and reformatting from scratch to [grade, theme, diff, condition, solution, answer, hint]
for i in range(len(problems_list)):
    problem = splitter(problems_list[i])
    d = get_problem_diff(problem)
    res = [get_problem_class(problem)] # start with class, adding all into the array
    res.append(problem[0])             # add theme
    res.append(d)                      # add difficulty
    res += problem[4:8]                # add condition-solution-answer-hint
    total.append(res)                  # ... and go for the next problem

## 3. SQL - database creation

In [11]:
try:
    pswd_file = open("C://Users//novle//Desktop//pswd.txt", "r", encoding = "utf8") # closed password
    pswd = pswd_file.read() # temporarily import all into one large string
except FileNotFoundError:
    print('You have no password, sorry...')

In [12]:
import mysql.connector # import to sql

mydb = mysql.connector.connect(
    host = "127.0.0.1",
    user = "root",
    password = pswd,
    database = "allproblems"
)

In [29]:
#mycursor = mydb.cursor(buffered=True) # use only if we need to clean everything stored!
#sql = "DROP TABLE prob2"
#mycursor.execute(sql)
#mydb.commit()

#mycursor = mydb.cursor() # create database
#sql = "CREATE TABLE prob2 (idproblem INT AUTO_INCREMENT PRIMARY KEY, grade INT, theme VARCHAR(100), diff INT, prcon VARCHAR(2000), solution VARCHAR(8500), answer VARCHAR(1500), hint VARCHAR(500))"
#mycursor.execute(sql)
#mydb.commit()

#mycursor = mydb.cursor(buffered = True) # import data into database
#sql = "INSERT INTO prob2 (grade, theme, diff, prcon, solution, answer, hint) VALUES (%s, %s, %s, %s, %s, %s, %s)"
#mycursor.executemany(sql, total)
#mydb.commit()

#print(mycursor.rowcount, "problems was inserted.")

1807 problems was inserted.


#### Future realisation of topics in SQL

In [30]:
# CAREFUL!
# if ever need to erase and restart again the same way, uncomment
# this will delete all existing tables!!

#mycursor = mydb.cursor(buffered=True)
#sql = "DROP TABLE topics"
#mycursor.execute(sql)
#mydb.commit()

#mycursor = mydb.cursor(buffered=True)
#sql = "DROP TABLE problem_topic"
#mycursor.execute(sql)
#mydb.commit()

#mycursor = mydb.cursor()
#sql = "CREATE TABLE topics (idtopic INT AUTO_INCREMENT PRIMARY KEY, topicname VARCHAR(150))"
#mycursor.execute(sql)
#mydb.commit()

#mycursor = mydb.cursor()
#sql = "CREATE TABLE problem_topic (idproblem INT, idtopic INT, CONSTRAINT prob_top PRIMARY KEY (idproblem, idtopic), CONSTRAINT con_problem FOREIGN KEY (idproblem) REFERENCES problems (idproblem), CONSTRAINT cor_topic FOREIGN KEY (idtopic) REFERENCES topics (idtopic))"
#mycursor.execute(sql)
#mydb.commit()

#mycursor = mydb.cursor()
#mycursor.execute("SELECT * FROM problems")
#myresult = mycursor.fetchall()

## 4. Exporting from SQL straight to pdf

In [13]:
# 1. select from DB by theme number
def problem_selector(pr_class, theme_num):
    mycursor = mydb.cursor(buffered=True)
    themes = themes_choose[pr_class-6]
    strreq = "SELECT * FROM prob2 WHERE theme = " + str("\'" + themes[theme_num] + "\'")
    mycursor.execute(strreq)
    problemset = mycursor.fetchall()
    mydb.commit()
    return problemset

# mode is 0 for conditions only, 1 for hints, 2 for answers + solutions
def latex_file_creator(theme = '', mode = 0, name = 'default', path = 'C://Users//novle//AppData//Local//MiKTeX//'):
    preamb = open("C://Users//novle//AppData//Local//MiKTeX//prgen.txt", "r", encoding = "utf8") # using this as preamble
    f_res = open(path + name + ".tex", "a", encoding = "utf8") # create file with chosen name (default.txt by default)
    initial = "\n\n" + r"\begin{document}" + "\n" + "\large" + "\n" + "\mode{" + str(mode) + "}" + "\n\n" + r"\today" + "\n"
    title = r"\begin{titlegen}{" + str(mode) + "}{" + theme + "}\end{titlegen}\n\n" # with chosen theme
    text = preamb.read() + initial + title # combine strings 
    f_res.write(text)                      # write into latex file
    preamb.close()
    f_res.close()

# Takes list of problems from db and adds them into file
def latex_problems_adder(problemset, name = 'default', path = 'C://Users//novle//AppData//Local//MiKTeX//'):
    f_res = open(path + name + ".tex", "a", encoding = "utf8") # with chosen name (default.txt by default)
    for elem in problemset:
        problem = r"\begin{problem}{" + elem[2] + "}{" + str(elem[1]) + "}{" + str(elem[0]) + "}{" + str(elem[3]) + "}\n{" + elem[4] + "}\n{" + elem[5] + "}\n{" + elem[6] + "}{" + elem[7] + "}\end{problem}\n\n"
        f_res.write(problem)
    f_res.write("\end{document}")
    f_res.close()

# produces pdf in any folder, default 'C://Users//novle//AppData//Local//MiKTeX//default.txt'
def latex_file_executor(name = 'default', path = 'C://Users//novle//AppData//Local//MiKTeX//'):
    os.chdir(path) # set path
    cmd = 'pdflatex ' + name + '.tex' # printing to pdf using pdflatex
    os.system(cmd)

# creates file, fills it, produces pdf
def universal_creator(problemset, theme = '', mode = 0, name = 'default', path = 'C://Users//novle//AppData//Local//MiKTeX//'):
    latex_file_creator(theme, mode, name, path)
    latex_problems_adder(problemset, name, path)
    latex_file_executor(name, path)

In [33]:
#def problem_selector2(pr_class, theme_num): # select by theme number (from tex)
#    mycursor = mydb.cursor(buffered=True)
#    themes = themes_choose[pr_class-6]
#    strreq = "SELECT * FROM problems WHERE theme = " + str("\'" + themes[theme_num] + "\'")
#    mycursor.execute(strreq)
#    problemset = mycursor.fetchall()
#    mydb.commit()
#    return problemset

### 4.1. Example of extracting:

In [14]:
contents = ['Id: ', 'Class: ', 'Theme: ', 'Difficulty: ', 'Condition: ', 'Solution: ', 'Answer: ', 'Hint: ']
clas = 8
theme = 6
selected_task = problem_selector(clas, theme)[0]
for i in range(8):
    print(contents[i], selected_task[i], '\n')    

Id:  819 

Class:  8 

Theme:  График квадратного корня. 

Difficulty:  0 

Condition:  Нарисовать график функции $y = 1 - 2\sqrt{x + 3}$. 

Solution:  \textcolor{Green}{\textbf{Ограничения:}} Так как подкоренное выражение должно быть неотрицательным, стоит отметить что $\, x + 3 \geqslant 0 \;\Rightarrow\; x \geqslant -3 $.\smallskip\\
Чтобы нарисовать данный график, надо понять, что этот график будет являться графиком функции $y = \sqrt{x}$, но с несколькими изменениями:\\
\begin{minipage}{\linewidth}
    \begin{minipage}{0.44\linewidth}
    1) Функция будет не возрастающей, а убывающей, так как перед корнем стоит знак минуса, поэтому график функции $y = \sqrt{x}$ отражается относительно оси абсцисс.\smallskip\\
    2) График функции будет сдвинут на $3$ влево по оси $x$ (так как под корнем стоит $x + 3$).
    \end{minipage}
    \hspace{0.05\linewidth}
    \begin{minipage}{0.5\linewidth}
    \begin{figure}[H] \includegraphics[width=\linewidth]{819_graph1.png} 
    \end{figure}
    \e

## 4.2. Extracting to pdf

In [15]:
# simply generates three separate problemset pdfs (only conditions; +hints; +answers and solutions)
def triple_creator(problemset, path = 'C://Users//novle//AppData//Local//MiKTeX//'):
    print('Введите название темы, которая вас интересует:')
    theme = input()
    print('Insert FileName (eng only, this would be the name in the file system):')
    common_name = input()
    for mode in range(3):
        name = common_name # produces name of file: example
        if mode == 1:
            name += '_with_hints' # produces name of file: example_with_hints
        if mode == 2:
            name += '_with_solutions' # produces name of file: example_with_solutions
        universal_creator(problemset, theme, mode, name, path) # create all three one by one

In [16]:
def smart_list(path = 'C://Users//novle//AppData//Local//MiKTeX//'): # asks questions iteratively to determine the result
    print('На какой класс готовим листок? (6-11)')
    go = True
    while go:
        try:
            pr_class = int(input())
            break
        except ValueError:
            print('You need to enter class as a number (6, 7, 8, 9, 10, 11 are accepted)')
    index0 = []
    while go:
        try:
            for i in range(len(themes_choose[pr_class-6])):
                index0.append(i+1)
            break
        except IndexError:
                print('You need to enter class as a number (6, 7, 8, 9, 10, 11 are accepted)')
                pr_class = int(input())
    res_var = pd.Series(data = themes_choose[pr_class-6], index = index0)
    print('... \nВот список всех доступных тем для выбора: \n', res_var, '\n')
    problemset = []
    while go:
        print('Введите число - номер темы, по которому нужно составить листок (0 to stop)')
        try:
            theme_num = int(input()) - 1 # choose number of interesting theme (-1 as it is choosen from the list which starts from 1) !!!!
            if theme_num != -1:
                problemset += problem_selector(pr_class, theme_num)
            else:
                go = False
        except ValueError:
            print('Введите номер существующей темы, а не название!')
            pass
        except IndexError:
            print('Темы с указанным номером не найдено. Введите номер существующей темы!')
            pass
    triple_creator(problemset, path) # produces problemset in three different modes separately
    print('Файл может быть найден по адресу ' + path)

### Realization:

In [17]:
smart_list()
# As an example:
# Class - 11
# Themes: 20 (then 0)
# Name: Calculating areas using definite integral
# AUC.tex

На какой класс готовим листок? (6-11)
11
... 
Вот список всех доступных тем для выбора: 
 1     Комплексные числа. Их свойства, арифметические...
2                                Комплексная плоскость.
3     Тригонометрическая форма записи комплексного ч...
4     Формула Муавра. Возведение комплексного числа ...
5           Квадратные уравнения для комплексных чисел.
6     Извлечение кубического корня из комплексного ч...
7     Извлечение корня произвольной степени из компл...
8     Решение кубических уравнений. Формула Кардано,...
9          Показательная функция, её свойства и график.
10                             Показательные уравнения.
11                           Показательные неравенства.
12                                 Свойства логарифмов.
13                           Логарифмические уравнения.
14                         Логарифмические неравенства.
15       Взятие производной для экспонент и логарифмов.
16                        Правила поиска первообразной.
17            